In [1]:
import torch

PROCESSAMENTO DE DADOS

Conferindo a existência do PDF e baixando-o caso não exista

In [2]:
import os
import requests

#Importando o pdf do STJ
pdf_path = "SumulasSTJ_Ramos.pdf"

#Dowload do PDF se o path não existir
if not os.path.exists(pdf_path):
    print("[INFO] O arquivo não existe, realizando o download...")

    #URL do PDF com todas as Súmulas
    url = "https://scon.stj.jus.br/docs_internet/jurisprudencia/tematica/download/SU/RamosDoDireito/SumulasSTJ_Ramos.pdf"

    filename = pdf_path

    #GET request pra URL
    response = requests.get(url)

    #Checa se a request deu certo
    if response.status_code == 200:
        #Abre o arquivo e salva
        with open(filename, "wb") as file: #wb = write binary
            file.write(response.content)
        print(f"[INFO] O arquivo foi baixado e salvo como {filename}.")
    else:
        print(f"[INFO] Falha no dowload do arquivo. Status code: {response.status_code}.")
#Se o path existe
else:
    print(f"O arquivo {pdf_path} existe.")

O arquivo SumulasSTJ_Ramos.pdf existe.


Abrindo o PDF!
Depois, formatando um pouco o documento e transformando-o em uma lista de dicionários

In [3]:
#Biblioteca para abrir o PDF (PyMUPDF, no GitHub)
import fitz
from tqdm.auto import tqdm #para barras de progresso

def text_formatter(text: str) -> str:
    #Faz pequenas formatações no texto do PDF
    cleaned_text = text.replace("\n", " ").strip()

    return cleaned_text

#Função para transformar o conteúdo+infos de todas as páginas em uma lista de dicionários
def open_and_read_pdf(pdf_path: str) -> list[dict]:
    #Abre o arquivo
    doc = fitz.open(pdf_path)
    #Cria uma lista vazia
    pages_and_texts = []

    #Itera sobre o PDF
    for page_number, page in tqdm(enumerate(doc)):
        #"Pega" o texto do documento
        text = page.get_text()
        #Formata de acordo com a função anterior
        text = text_formatter(text=text)
        #Coloca o dicionário referente a cada página na lista
        pages_and_texts.append({"page_number": page_number+1, #número da página
                               "page_char_count": len(text),  #contagem de caracteres
                               "page_word_count": len(text.split(" ")), #contagem brusca de palavras
                               "page_sentence_count_raw": len(text.split(". ")), #contagem brusca de frases
                               "page_token_count": len(text)/4, #1 token = ~4caracteres
                               "text": text}) #o texto em si
    
    #Retorna a lista dos dicionários das páginas
    return pages_and_texts
    
#Chamada da função
pages_and_text = open_and_read_pdf(pdf_path=pdf_path)

#teste
pages_and_text[:2]

0it [00:00, ?it/s]

[{'page_number': 1,
  'page_char_count': 630,
  'page_word_count': 106,
  'page_sentence_count_raw': 1,
  'page_token_count': 157.5,
  'text': 'SUMÁRIO SUMÁRIO DIREITO ADMINISTRATIVO - ÁGUA E ESGOTO 31 Súmula 407 31 Súmula 412 34 DIREITO ADMINISTRATIVO - ANISTIA POLÍTICA 36 Súmula 624 36 Súmula 647 39 DIREITO ADMINISTRATIVO - ATIVIDADE FARMACÊUTICA 44 Súmula 120 44 Súmula 275 47 Súmula 413 51 Súmula 561 54 DIREITO ADMINISTRATIVO - BEM PÚBLICO 61 Súmula 103 61 Súmula 619 64 DIREITO ADMINISTRATIVO - CÓDIGO DE TRÂNSITO BRASILEIRO 68 Súmula 127 68 Súmula 312 70 Súmula 434 74 Súmula 510 79 DIREITO ADMINISTRATIVO - CONCURSO PÚBLICO 85 Súmula 266 85 Súmula 377 88 Súmula 552 91 DIREITO ADMINISTRATIVO - DESAPROPRIAÇÃO 96 Súmula 12 96 Súmulas Organizadas por Ramo do Direito'},
 {'page_number': 2,
  'page_char_count': 561,
  'page_word_count': 108,
  'page_sentence_count_raw': 1,
  'page_token_count': 140.25,
  'text': 'Súmula 56 99 Súmula 67 102 Súmula 69 105 Súmula 70 108 Súmula 102 112 Súmula 

In [4]:
#teste
import random

random.sample(pages_and_text, k=3)

[{'page_number': 1549,
  'page_char_count': 2168,
  'page_word_count': 334,
  'page_sentence_count_raw': 22,
  'page_token_count': 542.0,
  'text': '"COMPETÊNCIA. JUNTA TRABALHISTA E JUIZ DE DIREITO INVESTIDO DE JURISDIÇÃO TRABALHISTA. COMPETÊNCIA DO TRIBUNAL REGIONAL DO TRABALHO AO QUAL OS JUÍZES ESTEJAM VINCULADOS. [...]" (CC 14574 CE, Rel. Ministro ANSELMO SANTIAGO, TERCEIRA SEÇÃO, julgado em 27/03/1996, DJ 13/05/1996, p. 15515) "CONFLITO DE COMPETÊNCIA. TRABALHISTA. LIMITES TERRITORIAIS. É DA COMPETÊNCIA DO RESPECTIVO TRT O JULGAMENTO DO CONFLITO ALUSIVO A LIMITES TERRITORIAIS, ENTRE JUÍZO ESTADUAL E JUNTA DE CONCILIAÇÃO E JULGAMENTO." (CC 13950 SP, Rel. Ministro RUY ROSADO DE AGUIAR, SEGUNDA SEÇÃO, julgado em 11/10/1995, DJ 08/04/1996, p. 10440) "CONFLITO ENTRE JUNTA DE CONCILIAÇÃO E JULGAMENTO E JUIZ DE DIREITO INVESTIDO DE JURISDIÇÃO TRABALHISTA. EM HIPÓTESE TAL, CABE AO TRIBUNAL REGIONAL DO TRABALHO A QUE AMBOS ESTÃO VINCULADOS DIRIMIR O CONFLITO. [...]" (CC 14024 PR, Rel. Mini

In [5]:
#Breakdown dos dados trabalhados
import pandas as pd

df = pd.DataFrame(pages_and_text)
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,text
0,1,630,106,1,157.50,SUMÁRIO SUMÁRIO DIREITO ADMINISTRATIVO - ÁGUA ...
1,2,561,108,1,140.25,Súmula 56 99 Súmula 67 102 Súmula 69 105 Súmul...
2,3,764,120,1,191.00,Súmula 646 201 DIREITO ADMINISTRATIVO - IMPROB...
3,4,658,117,1,164.50,DIREITO ADMINISTRATIVO - SERVIDOR PÚBLICO MILI...
4,5,539,102,1,134.75,Súmula 296 387 Súmula 297 389 Súmula 298 391 S...


In [6]:
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count
count,2621.00,2621.00,2621.00,2621.00,2621.00
mean,1311.00,2071.00,325.17,21.32,517.75
std,756.76,1073.82,170.49,15.83,268.46
min,1.00,68.00,8.00,1.00,17.00
25%,656.00,1240.00,195.00,4.00,310.00
50%,1311.00,2025.00,310.00,21.00,506.25
75%,1966.00,2946.00,466.00,34.00,736.50
max,2621.00,4870.00,805.00,70.00,1217.50


Tokens são importantes, pois:
1. Modelos de embedding não lidam com tokens infinitos;
2. LLMs não lidam com tokens infinitos.

O modelo de embedding usado neste projeto aceita o máximo de 384 tokens. A média de tokens por página do PDF é de 325.17

In [7]:
from spacy.lang.pt import Portuguese

nlp = Portuguese()

#Adiciona um segmentador de sentenças
sentencizer = nlp.add_pipe("sentencizer")
sentencizer.punct_chars = ["!", ".", "?", "..."] #adiciona a reticências como delimitador

#teste
doc = nlp("Isso é uma frase! Outra frase... Oi.")
assert len(list(doc.sents)) == 3
list(doc.sents)

[Isso é uma frase!, Outra frase..., Oi.]

In [8]:
for item in tqdm(pages_and_text):
    item["sentences"] = list(nlp(item["text"]).sents)

    #Para ter certeza que todas as sentenças serão strings
    item["sentences"] = [str(sentence) for sentence in item["sentences"]]

    #Contando as frases
    item["page_sentence_count_spacy"] = len(item["sentences"])

  0%|          | 0/2621 [00:00<?, ?it/s]

In [9]:
#teste
random.sample(pages_and_text, k=1)

[{'page_number': 2362,
  'page_char_count': 2332,
  'page_word_count': 335,
  'page_sentence_count_raw': 29,
  'page_token_count': 583.0,
  'text': '214.17. Súmula 334 SÚMULA 334 DIREITO TRIBUTÁRIO - ICMS Enunciado: O ICMS não incide no serviço dos provedores de acesso à Internet. Referências Legislativas: LEG:FED LEI:009472 ANO:1997 ART:00060 ART:00061 PAR:00001 LEG:FED LCP:000087 ANO:1996 ART:00002 Órgão Julgador: PRIMEIRA SEÇÃO Data da decisão: 13/12/2006 Fonte: DJ DATA:14/02/2007 PG:00246 RSSTJ VOL.:00028 PG:00159 RSTJ VOL.:00205 PG:00492 Excerto dos Precedentes Originários: "[...] ICMS. SERVIÇO DE PROVIMENTO DE ACESSO À INTERNET. SERVIÇO DE VALOR ADICIONADO. INCIDÊNCIA. DESCABIMENTO. [...] A 1ª Seção, em 11.05.2005, concluindo o julgamento do ERESP 456.650/PR, Rel. originário Min. José Delgado, Rel. p/ o acórdão Min. Franciulli Netto, firmou orientação no sentido da não-incidência do ICMS sobre os serviços prestados pelos provedores de acesso à Internet. Ressalva do entendimento p

In [10]:
df = pd.DataFrame(pages_and_text)
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,page_sentence_count_spacy
count,2621.00,2621.00,2621.00,2621.00,2621.00,2621.00
mean,1311.00,2071.00,325.17,21.32,517.75,20.83
std,756.76,1073.82,170.49,15.83,268.46,15.37
min,1.00,68.00,8.00,1.00,17.00,1.00
25%,656.00,1240.00,195.00,4.00,310.00,4.00
50%,1311.00,2025.00,310.00,21.00,506.25,21.00
75%,1966.00,2946.00,466.00,34.00,736.50,33.00
max,2621.00,4870.00,805.00,70.00,1217.50,61.00


In [11]:
#Transformando grupos de frases em "pedaços" menores
num_sentence_chunk_size = 8

#Função que divide listas de textos em "pedaços"
def split_list(input_list: list[str], slice_size: int=num_sentence_chunk_size) -> list[list[str]]:
    result = []
    for i in range(0, len(input_list), slice_size):
        chunk = input_list[i:i+slice_size] #slicing
        result.append(chunk)

    return result

#teste
test_list = list(range(25))
split_list(test_list)

[[0, 1, 2, 3, 4, 5, 6, 7],
 [8, 9, 10, 11, 12, 13, 14, 15],
 [16, 17, 18, 19, 20, 21, 22, 23],
 [24]]

In [12]:
#Dividindo "pages_and_text" em pedaços
for item in tqdm(pages_and_text):
    item["sentences_chunks"] = split_list(input_list=item["sentences"], slice_size=num_sentence_chunk_size)
    item["num_chunks"] = len(item["sentences_chunks"])

  0%|          | 0/2621 [00:00<?, ?it/s]

In [13]:
#teste
random.sample(pages_and_text, k =1)

[{'page_number': 1042,
  'page_char_count': 404,
  'page_word_count': 82,
  'page_sentence_count_raw': 1,
  'page_token_count': 101.0,
  'text': 'HC           483657  SC 2018/0331696-5 Decisão:07/02/2019 DJE DATA:15/02/2019 RHC           85339  SC 2017/0133329-0 Decisão:04/09/2018 DJE DATA:19/09/2018 RHC           90109  MG 2017/0255042-7 Decisão:19/03/2019 DJE DATA:25/03/2019 RHC           93154  SC 2017/0329840-4 Decisão:04/06/2019 DJE DATA:11/06/2019 DIREITO PENAL - APROPRIAÇÃO INDÉBITA TRIBUTÁRIA Súmula 658 scon.stj.jus.br/SCON/sumstj/ 1042',
  'sentences': ['HC           483657  SC 2018/0331696-5 Decisão:07/02/2019 DJE DATA:15/02/2019 RHC           85339  SC 2017/0133329-0 Decisão:04/09/2018 DJE DATA:19/09/2018 RHC           90109  MG 2017/0255042-7 Decisão:19/03/2019 DJE DATA:25/03/2019 RHC           93154  SC 2017/0329840-4 Decisão:04/06/2019 DJE DATA:11/06/2019 DIREITO PENAL - APROPRIAÇÃO INDÉBITA TRIBUTÁRIA Súmula 658 scon.stj.jus.br/SCON/sumstj/ 1042'],
  'page_sentence_count

In [14]:
df = pd.DataFrame(pages_and_text)
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,page_sentence_count_spacy,num_chunks
count,2621.00,2621.00,2621.00,2621.00,2621.00,2621.00,2621.00
mean,1311.00,2071.00,325.17,21.32,517.75,20.83,3.14
std,756.76,1073.82,170.49,15.83,268.46,15.37,1.81
min,1.00,68.00,8.00,1.00,17.00,1.00,1.00
25%,656.00,1240.00,195.00,4.00,310.00,4.00,1.00
50%,1311.00,2025.00,310.00,21.00,506.25,21.00,3.00
75%,1966.00,2946.00,466.00,34.00,736.50,33.00,5.00
max,2621.00,4870.00,805.00,70.00,1217.50,61.00,8.00


In [15]:
#Transformando cada pedaço em um item

import re

pages_and_chunks = []
for item in tqdm(pages_and_text):
    for sentence_chunk in item["sentences_chunks"]:
        chunk_dict = {}
        chunk_dict["page_number"] = item["page_number"]

        #Coloca as frases juntas numa estrutura de parágrafo
        joined_sentence_chunk = "".join(sentence_chunk).replace("  ", " ").replace("   ", " ").strip()
        joined_sentence_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk) #".A" => ". A" (maiúsculas)
        joined_sentence_chunk = re.sub(r'(\[.*?\])', r'\1 ', joined_sentence_chunk).replace("  ", " ")

        chunk_dict["sentence_chunk"] = joined_sentence_chunk

        chunk_dict["chunk_char_count"] = len(joined_sentence_chunk)
        chunk_dict["chunk_word_count"] = len([word for word in joined_sentence_chunk.split(" ")])
        chunk_dict["chunk_token_count"] = len(joined_sentence_chunk)/4

        pages_and_chunks.append(chunk_dict)

len(pages_and_chunks)

  0%|          | 0/2621 [00:00<?, ?it/s]

8220

In [16]:
random.sample(pages_and_chunks, k=1)

[{'page_number': 2588,
  'sentence_chunk': 'O manejo de mandado de segurança é capaz de interromper o prazo prescricional em relação à ação de repetição de indébito tributário [...] . Isso, nos termos do previsto no art. 202 do Código Civil, somente pode ocorrer uma vez.4. No caso, com a impetração do mandado de segurança em 10/12/1998, o prazo prescricional para a repetição do indébito foi interrompido e recomeçou a ser contado a partir do trânsito em julgado do provimento jurisdicional, perpetrado em 14/08/2002. Entretanto, tal ação somente foi ajuizada em 27/06/2008. Logo, a pretensão está fulminada pela prescrição. [...] " (REsp 1248618 SC, Rel. Ministro BENEDITO GONÇALVES, PRIMEIRA TURMA, julgado em 18/12/2014, DJe 13/02/2015) "[...] ',
  'chunk_char_count': 705,
  'chunk_word_count': 111,
  'chunk_token_count': 176.25}]

In [17]:
df = pd.DataFrame(pages_and_chunks)
df.describe().round(2)

,page_number,chunk_char_count,chunk_word_count,chunk_token_count
count,8220.00,8220.00,8220.00,8220.00
mean,1326.73,650.01,94.02,162.50
std,746.93,324.24,47.85,81.06
min,1.00,68.00,8.00,17.00
25%,692.00,418.00,60.00,104.50
50%,1305.00,605.00,89.00,151.25
75%,1970.00,837.00,122.00,209.25
max,2621.00,2287.00,353.00,571.75


Embedding

In [19]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2", device="cpu")

#Cria uma lista de frases (teste)
sentences = ["Usando a biblioteca Sentence Tranformer para fazer o embedding", 
            "O embedding pode acontecer de frase por frase ou a partir de uma lista.", 
            "Eu gosto de chocolate!"]

embeddings = embedding_model.encode(sentences)
embeddings_dict = dict(zip(sentences, embeddings))

for sentence, embedding in embeddings_dict.items():
    print(f"Setence: {sentence}")
    print(f"Embedding: {embedding}")
    print()

/mnt/d/Códigos/LIA/simple-local-rag/venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Setence: Usando a biblioteca Sentence Tranformer para fazer o embedding
Embedding: [-1.47836590e-02  1.90544818e-02 -9.99869499e-03  4.96543869e-02
 -2.43156347e-02  6.36120501e-04  1.96538102e-02  5.77246491e-03
 -9.17542540e-03 -5.11974506e-02  2.59894133e-02  5.05687259e-02
  1.51836984e-02  8.48388672e-03  6.82999892e-03 -2.98298951e-02
  1.86932050e-02  4.58472408e-02 -8.14705342e-03 -3.46248113e-02
  1.91803258e-02 -9.02365136e-05  4.93789986e-02 -1.03622081e-03
  5.34315519e-02 -2.14828900e-03  3.52117568e-02 -2.54522469e-02
  3.03164404e-02  3.11904121e-02 -3.23202424e-02 -2.04441715e-02
  4.22542058e-02 -1.10521847e-02  1.18529886e-06 -2.06530057e-02
 -8.46405327e-02  1.05508612e-02  1.55221333e-03 -4.95256633e-02
  2.94080377e-02 -5.22783734e-02  5.39313769e-03  2.29291478e-03
 -4.39947657e-02  1.33498653e-03  3.47713381e-03 -6.93790475e-03
  5.43317478e-03  3.41029204e-02 -2.18030158e-02 -7.99956992e-02
  2.28060707e-02  4.11186367e-02 -1.79969762e-02 -1.94076896e-02
 -3.448

In [20]:
embeddings[0].shape

(768,)

In [27]:
%%time

embedding_model.to("cpu")

#Faz embedding nas chunks uma por uma
for item in tqdm(pages_and_chunks):
    item["embedding"] = embedding_model.encode(item["sentence_chunk"])

  0%|          | 0/8220 [00:00<?, ?it/s]

CPU times: user 2h 43min 56s, sys: 5min 32s, total: 2h 49min 29s
Wall time: 42min 21s


In [24]:
%%time

text_chunks = [item["sentence_chunk"] for item in pages_and_chunks]
text_chunks[90]

CPU times: user 92.8 ms, sys: 215 μs, total: 93 ms
Wall time: 87.5 ms


'3.3. Súmula 413 SÚMULA 413 DIREITO ADMINISTRATIVO - ATIVIDADE FARMACÊUTICA Enunciado: O farmacêutico pode acumular a responsabilidade técnica por uma farmácia e uma drogaria ou por duas drogarias. Referências Legislativas: LEG:FED LEI:005869 ANO:1973 ***** CPC-73 CÓDIGO DE PROCESSO CIVIL DE 1973 ART:0543C LEG:FED LEI:005991 ANO:1973 ART:00020 LEG:FED DEC:074170 ANO:1974 ART:00028 LEG:FED RES:000008 ANO:2008 ART:00002 PAR:00001 (SUPERIOR TRIBUNAL DE JUSTIÇA - STJ) Órgão Julgador: PRIMEIRA SEÇÃO Data da decisão: 25/11/2009 Fonte: DJE DATA:16/12/2009 RSSTJ VOL.:00039 PG:00113 RSTJ VOL.:00217 PG:01194 Excerto dos Precedentes Originários: "[...] RECURSO ESPECIAL REPRESENTATIVO DE CONTROVÉRSIA. ART. 543-C, DO CPC. ADMINISTRATIVO. CONSELHO REGIONAL DE FARMÁCIA.'

In [25]:
len(text_chunks)

8220

In [32]:
#Salvar os embeddings num arquivo
text_chunks_and_embeddings_df = pd.DataFrame(pages_and_chunks)
embeddings_df_save_path = "text_chunks_and_embeddings_df.csv"
text_chunks_and_embeddings_df.to_csv(embeddings_df_save_path, index=False)

In [33]:
#Importar o arquivo salvo e visualizar
text_chunks_and_embeddings_df_load = pd.read_csv(embeddings_df_save_path)
text_chunks_and_embeddings_df_load.head()

,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,1,SUMÁRIO SUMÁRIO DIREITO ADMINISTRATIVO - ÁGUA ...,630,106,157.50,[-3.87285240e-02 9.39896181e-02 7.45087815e-...
1,2,Súmula 56 99 Súmula 67 102 Súmula 69 105 Súmul...,561,108,140.25,[-3.07321530e-02 9.78185907e-02 -1.08964285e-...
2,3,Súmula 646 201 DIREITO ADMINISTRATIVO - IMPROB...,764,120,191.00,[-1.26071302e-02 1.00405850e-01 -1.52930683e-...
3,4,DIREITO ADMINISTRATIVO - SERVIDOR PÚBLICO MILI...,658,117,164.50,[ 3.98039864e-03 6.20249920e-02 -1.73659157e-...
4,5,Súmula 296 387 Súmula 297 389 Súmula 298 391 S...,539,102,134.75,[-4.35576811e-02 9.61721987e-02 -2.87861302e-...


RAG - BUSCAS E RESPOSTAS

In [ ]:
import random

import torch
import numpy as np
import pandas as pd

device = "cuda" if torch.cuda.is_available() else "cpu"

text_chunks_and_embeddings_df = pd.read_csv("text_chunks_and_embeddings_df.csv")

#Converte a coluna dos embeddings para np.array
text_chunks_and_embeddings_df["embedding"] = text_chunks_and_embeddings_df["embedding"].apply(lambda x: np.fromstring(x.strip("[]"), sep=" "))

#Converte texts and embeddings df para uma lista de dicionários
pages_and_chunks = text_chunks_and_embeddings_df.to_dict(orient="records")

text_chunks_and_embeddings_df

In [35]:
text_chunks_and_embeddings_df['embedding']

0       [-3.87285240e-02  9.39896181e-02  7.45087815e-...
1       [-3.07321530e-02  9.78185907e-02 -1.08964285e-...
2       [-1.26071302e-02  1.00405850e-01 -1.52930683e-...
3       [ 3.98039864e-03  6.20249920e-02 -1.73659157e-...
4       [-4.35576811e-02  9.61721987e-02 -2.87861302e-...
                              ...                        
8215    [ 6.74787152e-04  6.66686893e-02  2.15502642e-...
8216    [-6.33772579e-04  3.38131711e-02  9.83670447e-...
8217    [ 2.65831072e-02 -8.75060931e-02 -3.57318041e-...
8218    [ 1.02295931e-02  2.48035900e-02 -5.80333965e-...
8219    [ 2.21385248e-02 -2.23043337e-02 -1.99836958e-...
Name: embedding, Length: 8220, dtype: object

In [ ]:
embeddings = text_chunks_and_embeddings_df["embedding"].tolist()